Sources

#### Projet End-to-End (Pas NHL API)
https://medium.com/coinmonks/4-718-using-machine-learning-to-bet-on-the-nhl-25d16649cd52

#### Projet Live Feed (NHL API)
##### Utilise le live feed pour shot sur gardien
https://towardsdatascience.com/nhl-analytics-with-python-6390c5d3206d

#### Docu NHL API
https://github.com/dword4/nhlapi

Ancienne Doc:
https://webcache.googleusercontent.com/search?q=cache:ILYmzDuxW70J:https://github.com/dword4/nhlapi+&cd=1&hl=en&ct=clnk&gl=ca

In [5]:
# Check Python Version
from platform import python_version
import pandas as pd
import requests  
from pandas.io.json import json_normalize
import datetime

pd.set_option('display.max_columns', None)

#print(pd.show_versions())

### Import teams data

In [7]:
url = 'https://statsapi.web.nhl.com/api/v1/teams'
r = requests.get(url)
json_data = r.json()
teams = pd.json_normalize(json_data['teams'], sep = "_")

teams.to_csv('../../data/teams.csv')